# Tutorial for batch hyper parameter tuing using Quante Carlo

In [ ]:
import pandas as pd
from quante_carlo import hp_tune, worker
import time
import numpy as np
import multiprocessing as mp
from sklearn.datasets import load_diabetes
import requests
import json
import matplotlib.pyplot as plt

In [ ]:
data = load_diabetes()

In [ ]:
hp_ranges = [[.01, .99], [.01, .99]]
hp_types = ['float', 'float']
gbatch_size = 100 # how many points to evaluate in bayesian optimization
g_procs = 2 # how many requests to bayesian optimizer
n_procs = 4 # how many processors am i running, bayesian optimizer will give you this many 'next points'
n_iter = 50
oparameters = {'model': 'ElasticNet',
               'data': data}
logfile_name = 'log.txt'

In [ ]:
if __name__ == '__main__':
    
    enet_tuner = hp_tune.session(worker.regression_worker,
                                 hp_ranges=hp_ranges, 
                                 batch_sz=gbatch_size, n_gpr_processors=g_procs, use_qc='True',
                                 n_processors=n_procs, n_iter=n_iter, 
#                                 bo_url='http://localhost:8000', # if you're running the optimizer locally
                                 bo_url='https://boaz.onrender.com',
                                 other_parameters = oparameters, log_file=logfile_name)

    p = mp.Pool()
    start = time.time()    
    session = enet_tuner.tune(p)

    print("{} seconds".format(round(time.time() - start,2)))
    p.close()
    

In [ ]:
plt.plot(enet_tuner.summary()[['iteration', 'score']].groupby('iteration').max())

In [ ]:
enet_tuner.summary()